In [76]:
from k8sclient.dataframes import collect_all_endpoints

_endpoint_infos, endpoint_ip_ports = collect_all_endpoints()

port_list = endpoint_ip_ports[(endpoint_ip_ports.protocol == 'TCP') & (endpoint_ip_ports.target_kind == 'Pod')][['ip', 'port', "node_name", "target_namespace", "target_name"]]
port_list = port_list.drop_duplicates()
# b.groupby(["node_name", "ip"]).agg(lambda x: x.sample()).reset_index()

from k8sclient.keywords import (
    list_ready_nodes,
    cleanup_pods,
    get_pod_ip,
    pod_exec,
    is_pod_running,
)
from k8sclient.K8SClient import (
    PodBuilder,
)
import time

nodes = list_ready_nodes()
namespace = "health-check"
image = "127.0.0.1:30100/library/memcached:check2"
args = "memcached -m 1028 -u root -v"
# node_marks = {n: "-".join(n.split(".")) for n in nodes}


def _wait_for_pod_ready(timeout=420):
    ready = [False for _ in nodes]
    start = time.time()
    while True:
        if all(ready):
            break
        for i, n in enumerate(nodes):
            if ready[i]:
                continue
            pod_name = pod_names[n]
            ready[i] = is_pod_running(namespace, pod_name)
            if ready[i]:
                print pod_name, "is Running."
        if time.time() - start > timeout:
            break
        time.sleep(3)
    return [nodes[i] for i, r in enumerate(ready) if r]


pod_names = {}
for n in nodes:
    node_mark = "-".join(n.split("."))
    pod_name = "check-pod-on-%s" % (node_mark, )
    pod_names[n] = pod_name
    pod = PodBuilder(
        pod_name,
        namespace,
    ).set_node(
        n
    ).add_container(
        pod_name,
        image=image,
        args=args,
        requests={'cpu': '0', 'memory': '0'}
    )
    # pod.deploy()


ready_nodes = _wait_for_pod_ready()


def _check(pod, server, port):
    return pod_exec(namespace, pod, ["/opt/check2.sh", str(server), str(port)])

bads = []
#for n in ready_nodes:
#    print "checking on", n
#    for index, row in port_list.iterrows():
#        # print row['ip'], row['port'], row['node_name']
#        r = _check(pod_names[n], row['ip'], row['port'])
#        if not r:
#            bads.append(row)
#    break

for index, row in port_list.iterrows():
    r = _check(pod_names[row['node_name']], row['ip'], row['port'])
    if not r:
        bads.append(row)

check-pod-on-10-19-137-140 is Running.
check-pod-on-10-19-137-141 is Running.
check-pod-on-10-19-137-142 is Running.
check-pod-on-10-19-137-143 is Running.
check-pod-on-10-19-137-144 is Running.
check-pod-on-10-19-137-145 is Running.
check-pod-on-10-19-137-146 is Running.
check-pod-on-10-19-137-147 is Running.
check-pod-on-10-19-137-148 is Running.
check-pod-on-10-19-137-149 is Running.
check-pod-on-10-19-137-150 is Running.
check-pod-on-10-19-137-151 is Running.
check-pod-on-10-19-137-152 is Running.
check-pod-on-10-19-137-153 is Running.
check-pod-on-10-19-137-154 is Running.
check-pod-on-10-19-140-12 is Running.
check-pod-on-10-19-140-15 is Running.
check-pod-on-10-19-140-4 is Running.


In [95]:
import pandas as pd
a = pd.concat(bads, axis=1, ignore_index=True).T[["target_namespace", "target_name","ip", "port"]]
a = a[a.port != 2888]
a.groupby(["target_namespace", "target_name","ip"]).agg(lambda x: ",".join([str(v) for v in x]))
# for i in a.groupby("target_namespace").count().index:
#     print i
#print type(bads[0]), len(bads)

port
target_namespace          target_name                            ip                          
bocom                     tf-single-node-gpu-2056935168-5b1zf    172.2.208.15            8888
cc-lei                    wordpress-1422378352-0hzmh             172.5.32.32               80
cc-yzx-dev                nginx-0                                172.6.48.7              3306
                          nginx-1                                172.4.16.24             3306
                          nginx-2                                172.3.64.8              3306
                          nginx-3                                172.2.80.9              3306
ccc-hadoop0               demo-namenode1-1293401286-4mbdx        172.5.32.53       8019,50470
                          demo-namenode2-3287128873-5z7lz        172.5.32.54            50470
                          demo-resourcemanager1-2551448957-d557m 172.5.32.33            19888
                          demo-resourcemanager2-2386429362-d3ngf 172.5.32.63   8031,8032,8030
dataflow-demo-ye          emqttd-node1-1495700831-n0s3v          172.6.48.9    6002,6001,6000
                          emqttd-node2-3470497125-094n1          172.5.160.23  6002,6001,6000
                          emqttd-node3-1151309163-s2274          172.4.16.32   6002,6001,6000
                          middlemanager-1537696651-194r1         172.2.208.9   8100,8102,8101
                          middlemanager-1537696651-d66c7         172.5.160.16  8100,8102,8101
                          middlemanager-1537696651-j2ngf         172.4.224.16       8102,8101
                          pre1-namenode1-130780127-wg6vq         172.4.16.23       8019,50470
                          pre1-namenode2-3681259272-p3lrk        172.4.16.22       8019,50470
                          pre1-resourcemanager1-246800627-h4bxz  172.5.32.25   8031,8032,8030
                          pre1-zookeeper2-1358308814-0rn63       172.2.80.53             3888
liye-test-sh              pre1-hmaster1-1436454217-lt658         172.5.160.7      16010,16000
monitor-essential-service elasticsearch-data-3bkmm               172.2.80.27        9200,9300
                          k1-rgjl1                               172.2.0.8                 22
                          k2-5342z                               172.2.112.4               22
                          k3-3dp3b                               172.2.48.7                22
monitor-system-graph      monitoring-grafana-1nzxz               172.6.80.11             3000
monitor-system-security   enn-monitor-security-token-web-42jq3   172.6.80.5              8080
patricktest               disktest-3tmlf                         172.4.16.27               80
                          disktest-7975q                         172.4.16.10               80
                          disktest-hl2bc                         172.2.80.21               80
shared-hdfs               pre1-namenode1-2391644360-qkkl2        172.5.32.24            50470
                          pre1-namenode2-3096419314-0rcjw        172.5.32.26       8019,50470
wj-itg-test               appsss1-0                              172.4.16.35             8080
wst-test                  k1-sqg2s                               172.2.0.10                22
                          k2-43psj                               172.2.112.20              22
                          k3-ml8mm                               172.2.48.11               22

In [3]:
addresses

,endpoint_id,hostname,ip,node_name,target_kind,target_name,target_namespace,target_uid
0,8325f3f4-2be7-11e7-9440-9c7da38258d7,None,172.2.112.9,10.19.137.142,Pod,graphite-985m3,monkey,6893b081-6773-11e7-b5d0-1866da19caf3
1,8328fa59-2be7-11e7-9440-9c7da38258d7,None,172.2.112.9,10.19.137.142,Pod,graphite-985m3,monkey,6893b081-6773-11e7-b5d0-1866da19caf3


In [4]:
ports

,endpoint_id,name,port,protocol
0,8325f3f4-2be7-11e7-9440-9c7da38258d7,udpport,8125,UDP
1,8328fa59-2be7-11e7-9440-9c7da38258d7,httpport,80,TCP


In [27]:
import pandas as pd
pd.merge(addresses, ports, on="endpoint_id", how='outer')

,endpoint_id,hostname,ip,node_name,target_kind,target_name,target_namespace,target_uid,name,port,protocol
0,8325f3f4-2be7-11e7-9440-9c7da38258d7,None,172.2.112.9,10.19.137.142,Pod,graphite-985m3,monkey,6893b081-6773-11e7-b5d0-1866da19caf3,udpport,8125,UDP
1,8328fa59-2be7-11e7-9440-9c7da38258d7,None,172.2.112.9,10.19.137.142,Pod,graphite-985m3,monkey,6893b081-6773-11e7-b5d0-1866da19caf3,httpport,80,TCP
